# Work for best score on XGB

In [1]:
import os,sys,time,random,math,time
import tarfile, zipfile

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.preprocessing import MinMaxScaler
import itertools

from sklearn.cross_validation import train_test_split, StratifiedShuffleSplit


import matplotlib.pyplot as plt
from IPython.display import display, Image

import xgboost as xgb

use_xgb=True #disable for speed

from subprocess import check_output
datadir="./input/"
print(check_output(["ls", datadir]).decode("utf8"))

%matplotlib inline  


test.csv
test.csv.zip
train.csv
train.csv.zip



In [2]:
## loading packages 
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.cross_validation import KFold
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing
import gc


In [3]:
def loadData(datadir,filename):
    # Load the wholesale customers dataset
    #data = pd.read_csv(filename)
    data = ''
    print ("loading: "+datadir+filename)
    try:
        if zipfile.is_zipfile(datadir+filename):
            z = zipfile.ZipFile(datadir+filename)
            filename = z.open(filename[:-4])
        else:
            filename=datadir+filename
        data = pd.read_csv(filename, parse_dates=True)  
        print ("Dataset has {} samples with {} features each.".format(*data.shape))
    except Exception as e:
        print ("Dataset could not be loaded. Is the dataset missing?")
        print(e)
    return data

def writeData(data,filename):
    # Load the wholesale customers dataset
    try:
        data.to_csv(filename, index=False)
    except Exception as e:
        print ("Dataset could not be written.")
        print(e)
    verify=[]
    try:
        with open(filename, 'r') as f:
            for line in f:
                verify.append(line)
        f.closed
        return verify[:5]
    except IOError:
        sys.std
        

In [4]:

def LabelEncoder(data):
    # lifted in parts from:
    #https://www.kaggle.com/mmueller/allstate-claims-severity/yet-another-xgb-starter/code
    features = data.columns
    cats = [feat for feat in features if 'cat' in feat]
    for feat in cats:
        data[feat] = pd.factorize(data[feat], sort=True)[0]
    return data

In [5]:
def kmeansPlusmeanshift(data):  # used the one above to get the # of clusters, using this for speed
    start_time = time.time()
    # use the new clusters number to predict each locations cluster
    print "\nredo kmeans with new cluster number from meanshift +1 to account for sampling..."
    k_means =KMeans(n_clusters=80,n_jobs=12)
    final_clusters=k_means.fit_predict(data.values)
    print("kmeans round 2 time:{}s".format(round((time.time()-start_time), 3) ))
    return final_clusters

## Load and process data

In [6]:
data = loadData(datadir,'train.csv.zip')
display(data.info())
display(data.head(5))

test_data= loadData(datadir,'test.csv.zip') 
display(test_data.info())
display(test_data.head(5))

loading: ./input/train.csv.zip
Dataset has 188318 samples with 132 features each.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188318 entries, 0 to 188317
Columns: 132 entries, id to loss
dtypes: float64(15), int64(1), object(116)
memory usage: 189.7+ MB


None

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


loading: ./input/test.csv.zip
Dataset has 125546 samples with 131 features each.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125546 entries, 0 to 125545
Columns: 131 entries, id to cont14
dtypes: float64(14), int64(1), object(116)
memory usage: 125.5+ MB


None

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,4,A,B,A,A,A,A,A,A,B,...,0.281143,0.466591,0.317681,0.61229,0.34365,0.38016,0.377724,0.369858,0.704052,0.392562
1,6,A,B,A,B,A,A,A,A,B,...,0.836443,0.482425,0.443760,0.71330,0.51890,0.60401,0.689039,0.675759,0.453468,0.208045
2,9,A,B,A,B,B,A,B,A,B,...,0.718531,0.212308,0.325779,0.29758,0.34365,0.30529,0.245410,0.241676,0.258586,0.297232
3,12,A,A,A,A,B,A,A,A,A,...,0.397069,0.369930,0.342355,0.40028,0.33237,0.31480,0.348867,0.341872,0.592264,0.555955
4,15,B,A,A,A,A,B,A,A,A,...,0.302678,0.398862,0.391833,0.23688,0.43731,0.50556,0.359572,0.352251,0.301535,0.825823


In [7]:
# combine the two frames so we can encode the labels!
test_data['loss']=0

lengthofData=len(data)
lengthoftest_data=len(test_data)

print("data:",lengthofData)
print("test:",lengthoftest_data)

combineddata=pd.concat([data,test_data])
lengthofcombined=len(combineddata)
print("combined:",lengthofcombined)

('data:', 188318)
('test:', 125546)
('combined:', 313864)


In [8]:
# the categorical data that we need in a number format
combineddata=LabelEncoder(combineddata)
print("label encoded")

label encoded


In [9]:
hold_columns=combineddata[['loss','id']] #don't want to mess with these

combineddata.drop(['loss','id'],1,inplace=True) 
columns=combineddata.columns #need these to recreate the DF

#scale the columns we see
scaler= MinMaxScaler()   
values = scaler.fit_transform(combineddata.values)
combineddata= pd.DataFrame(values, columns=columns)

#put these back on for the moment!
combineddata['loss']=hold_columns['loss'].tolist()
combineddata['id']=hold_columns['id'].tolist()
del hold_columns
del values
del scaler
print "Data scaled"

Data scaled


In [10]:
# taken from Ali's script (https://www.kaggle.com/aliajouz/allstate-claims-severity/singel-model-lb-1117)
combineddata["cont1"] = np.sqrt(combineddata["cont1"])
combineddata["cont4"] = np.sqrt(combineddata["cont4"])
combineddata["cont5"] = np.sqrt(combineddata["cont5"])
combineddata["cont8"] = np.sqrt(combineddata["cont8"])
combineddata["cont10"] = np.sqrt(combineddata["cont10"])
combineddata["cont11"] = np.sqrt(combineddata["cont11"])
combineddata["cont12"] = np.sqrt(combineddata["cont12"])

combineddata["cont6"] = np.log(combineddata["cont6"] + 0000.1)
combineddata["cont7"] = np.log(combineddata["cont7"] + 0000.1)
combineddata["cont9"] = np.log(combineddata["cont9"] + 0000.1)
combineddata["cont13"] = np.log(combineddata["cont13"] + 0000.1)
combineddata["cont14"] = (np.maximum(combineddata["cont14"] - 0.179722, 0) / 0.665122) ** 0.25

In [11]:
display(data.info())
display(data.head(3))
# time to split the data back apart!
data=combineddata.iloc[:lengthofData].copy()
test_data=combineddata.iloc[lengthofData:].copy()
x_test_data=test_data.drop(['loss','id'],1) .values# didn't have the loss column before, make it go away! don't need ID!


# we don't want the ID columns in X
x=data.drop(['id','loss'],1).values
# loss is our label
y=data['loss'].values

print("Pre-Processing done")
display(data.info())
display(data.head(3))
print("data:",len(x))
print("labels:",len(y))
print("test:",len(x_test_data))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188318 entries, 0 to 188317
Columns: 132 entries, id to loss
dtypes: float64(15), int64(1), object(116)
memory usage: 189.7+ MB


None

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09


Pre-Processing done
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188318 entries, 0 to 188317
Columns: 132 entries, cat1 to id
dtypes: float64(131), int64(1)
memory usage: 189.7 MB


None

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,...,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss,id
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-0.953493,0.296826,-0.255633,0.916140,0.744792,0.761787,-0.070392,0.984628,2213.18,1
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,-0.704206,0.698552,-0.792214,0.664384,0.560798,0.585682,-0.330645,0.343682,1283.60,2
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,-1.009448,0.220662,-1.016372,0.571049,0.599347,0.591963,-1.211326,1.018094,3005.09,5


('data:', 188318)
('labels:', 188318)
('test:', 125546)


### alt preprocess

In [18]:

### setup
ID = 'id'
TARGET = 'loss'
NFOLDS = 5
SEED = 1
NROWS = None
DATA_DIR = "./input"
OUT_DIR ="./output"


### reading data 
TRAIN_FILE = "{0}/train.csv".format(DATA_DIR)
TEST_FILE = "{0}/test.csv".format(DATA_DIR)
SUBMISSION_FILE = "{0}/sample_submission.csv".format(DATA_DIR)

train = pd.read_csv(TRAIN_FILE, nrows=NROWS)
test = pd.read_csv(TEST_FILE, nrows=NROWS)



ntrain = train.shape[0]
ntest = test.shape[0]
y = np.log(train[TARGET]).ravel()
id_train= train[ID]
id_test= test[ID]

train_test = pd.concat((train, test)).reset_index(drop=True)

### remeber the order
train_test[ID]=pd.Categorical(train_test[ID], train_test[ID].values.tolist())

### factorize
cats = [feat for feat in train.columns if 'cat' in feat]
for cat in cats:
    sorting_list=np.unique(sorted(train_test[cat],key=lambda x:(str.lower(x),x)))
    train_test[cat]=pd.Categorical(train_test[cat], sorting_list)
    train_test=train_test.sort_values(cat)
    train_test[cat] = pd.factorize(train_test[cat], sort=True)[0]

### reorder 
train_test=train_test.sort_values(ID)
gc.collect()


#### preprocessing
train_test["cont1"] = np.sqrt(preprocessing.minmax_scale(train_test["cont1"]))
train_test["cont4"] = np.sqrt(preprocessing.minmax_scale(train_test["cont4"]))
train_test["cont5"] = np.sqrt(preprocessing.minmax_scale(train_test["cont5"]))
train_test["cont8"] = np.sqrt(preprocessing.minmax_scale(train_test["cont8"]))
train_test["cont10"] = np.sqrt(preprocessing.minmax_scale(train_test["cont10"]))
train_test["cont11"] = np.sqrt(preprocessing.minmax_scale(train_test["cont11"]))
train_test["cont12"] = np.sqrt(preprocessing.minmax_scale(train_test["cont12"]))

train_test["cont6"] = np.log(preprocessing.minmax_scale(train_test["cont6"])+0000.1)
train_test["cont7"] = np.log(preprocessing.minmax_scale(train_test["cont7"])+0000.1)
train_test["cont9"] = np.log(preprocessing.minmax_scale(train_test["cont9"])+0000.1)
train_test["cont13"] = np.log(preprocessing.minmax_scale(train_test["cont13"])+0000.1)
train_test["cont14"]=(np.maximum(train_test["cont14"]-0.179722,0)/0.665122)**0.25


### define x_train, x_test
train_test.drop([ID, TARGET], axis=1, inplace=True)
x = np.array(train_test.iloc[:ntrain,:])
x_test_data = np.array(train_test.iloc[ntrain:,:])
print("{},{}".format(x.shape, x_test_data.shape))



(188318, 130),(125546, 130)


### set up test/train split

In [19]:
#  train/validation split
X_train, X_validation, y_train, y_validation = train_test_split( x,
                                                                y_log,
                                                               test_size=0.20,
                                                                random_state=42)
display("sample train data size:{}".format(len(y_train)))

'sample train data size:150654'

### XGB Cross Validation on train

In [22]:
# XGB!
dtrain = xgb.DMatrix(X_train, label=y_train)
#params from:
#https://www.kaggle.com/mnabaee/allstate-claims-severity/labelencoding-and-xgb-cv/discussion
xgb_params = {
    'seed': 0,
    'colsample_bytree': 0.3085,
    'silent': 1,
    'subsample': 0.7,
    'learning_rate': 0.01,
    'objective': 'reg:linear',
    'max_depth': 10,
    'num_parallel_tree': 1,
    'min_child_weight': 4.2922,
    'eval_metric': 'mae',
    'eta':0.001,
    'gamma': 0.5290,
    'subsample':0.9930,
    'max_delta_step':0,
    'booster':'gbtree',
    'nrounds': 1001
}

start_time = time.time()
res = xgb.cv(xgb_params, dtrain, num_boost_round=2001, nfold=4, seed=42, stratified=False,
             early_stopping_rounds=50, verbose_eval=100, show_stdv=True, maximize=False)
print("CV time:{}s".format(round((time.time()-start_time), 3) ))

best_nrounds = res.shape[0] - 1
cv_mean = res.iloc[-1, 0]
cv_std = res.iloc[-1, 1]
print('CV-Mean: {0}+{1}'.format(cv_mean, cv_std))
print('CV-Mean(de-logged): {0}+{1}'.format(np.exp(cv_mean),np.exp( cv_std)))


[0]	train-mae:7.11322+0.00189649	test-mae:7.11322+0.00571393
[100]	train-mae:2.60584+0.000759059	test-mae:2.6058+0.00451613
[200]	train-mae:0.983527+0.000639137	test-mae:0.985647+0.00404663
[300]	train-mae:0.51959+0.000662105	test-mae:0.531452+0.00292577
[400]	train-mae:0.422357+0.000372575	test-mae:0.443676+0.00221396
[500]	train-mae:0.397458+0.00042104	test-mae:0.426088+0.00194746
[600]	train-mae:0.386379+0.000370325	test-mae:0.421041+0.00187737
[700]	train-mae:0.378835+0.000394337	test-mae:0.418803+0.00185872
[800]	train-mae:0.372785+0.000488224	test-mae:0.417518+0.00187112
[900]	train-mae:0.367784+0.000455709	test-mae:0.416647+0.001875
[1000]	train-mae:0.363389+0.000537295	test-mae:0.416071+0.00187202
[1100]	train-mae:0.359461+0.000319047	test-mae:0.415637+0.00186553
[1200]	train-mae:0.35602+0.000345769	test-mae:0.415342+0.00183219
[1300]	train-mae:0.352801+0.000294772	test-mae:0.415071+0.00184279
[1400]	train-mae:0.349816+0.00036471	test-mae:0.414868+0.00184587
[1500]	train-mae:0.

### actually train(XGB) on the data

In [23]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_validation)
start_time = time.time()
gbdt=xgb.train(xgb_params, dtrain, best_nrounds)
print("-XGB train time:{}s".format(round((time.time()-start_time), 3) ))
# now do a prediction and spit out a score(MAE) that means something
start_time = time.time()
curr_predict=gbdt.predict(dtest)
MAE=np.mean(abs(np.exp(curr_predict) - np.exp(y_validation)))
print("XGB Mean abs error: {:.2f}".format(MAE))
print("-XGB predict time:{}s".format(round((time.time()-start_time), 3) ))

#best so far:1197

-XGB train time:249.601s
XGB Mean abs error: 1129.19
-XGB predict time:2.527s


In [16]:
### setup training functions
ntrain = X_train.shape[0]
ntest = X_validation.shape[0]
kf = KFold(ntrain, n_folds=NFOLDS, shuffle=True, random_state=SEED)



class XgbWrapper(object):
    def __init__(self, seed=0, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 250)

    def train(self, x, y):
        dtrain = xgb.DMatrix(x, label=y)
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)

    def predict(self, x):
        return self.gbdt.predict(xgb.DMatrix(x))

def get_oof(clf):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = X_train[train_index]
        y_tr = y_train[train_index]
        x_te = X_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(X_validation)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

#### xgb
xgb_params = {
    'seed': 1,
    'colsample_bytree': 0.3085,
    'subsample': 0.9930,
    'eta': 0.1,
    'gamma': 0.5290,
    'booster' :  'gbtree',    
    'objective': 'reg:linear',
    'max_depth': 7,
    'min_child_weight': 4.2922,
    'eval_metric': 'mae'
}


# LB 1117.65579
#xgb_params['nrounds']=xgb2_rounds
xgb_params['nrounds']=493
print(xgb_params)

xgb_model = XgbWrapper(seed=SEED, params=xgb_params)
xgb_oof_train, xgb_oof_test = get_oof(xgb_model)

y_train_unlog=np.exp(y_train)
xgb_oof_train_unlog=pd.DataFrame(np.exp(xgb_oof_train))

display(len(y_train_unlog),y_train_unlog[3])
display(xgb_oof_train_unlog.info,xgb_oof_train_unlog.head(3))

print("XGB-CV: {}".format(mean_absolute_error(y_train_unlog,xgb_oof_train_unlog.values)))

xgb_tes
    

{'eval_metric': 'mae', 'seed': 1, 'nrounds': 493, 'booster': 'gbtree', 'colsample_bytree': 0.3085, 'min_child_weight': 4.2922, 'subsample': 0.993, 'eta': 0.1, 'objective': 'reg:linear', 'max_depth': 7, 'gamma': 0.529}


150654

inf

<bound method DataFrame.info of           0
0       inf
1       inf
2       inf
3       inf
4       inf
5       inf
6       inf
7       inf
8       inf
9       inf
10      inf
11      inf
12      inf
13      inf
14      inf
15      inf
16      inf
17      inf
18      inf
19      inf
20      inf
21      inf
22      inf
23      inf
24      inf
25      inf
26      inf
27      inf
28      inf
29      inf
...     ...
150624  inf
150625  inf
150626  inf
150627  inf
150628  inf
150629  inf
150630  inf
150631  inf
150632  inf
150633  inf
150634  inf
150635  inf
150636  inf
150637  inf
150638  inf
150639  inf
150640  inf
150641  inf
150642  inf
150643  inf
150644  inf
150645  inf
150646  inf
150647  inf
150648  inf
150649  inf
150650  inf
150651  inf
150652  inf
150653  inf

[150654 rows x 1 columns]>

,0
0,inf
1,inf
2,inf


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

### Retrain on all the data data, then predict the test set

In [ ]:
dtrain = xgb.DMatrix(x, label=y)
start_time = time.time()
gbdt=xgb.train(xgb_params, dtrain, best_nrounds)
print("XGB re-train time:{}s".format(round((time.time()-start_time), 3) ))


# now do a prediction and spit out a score(MAE) that means something
#
#print("Mean abs error: {:.2f}".format(np.mean(abs(cache[i+1] - y_test))))



In [ ]:
dtest = xgb.DMatrix(x_test_data)
start_time = time.time()
test_data['loss']=gbdt.predict(dtest)
print("XGB predict time:{}s".format(round((time.time()-start_time), 3) ))
result=test_data[['id','loss',]]
output_fname="result_submission_xgb.csv"
display(writeData(result,output_fname))



In [ ]:
#let's have a look at the std of the result, as a cross check
print("result std:",result.std(axis=0))

### EOF